### Finalise DataFrame

This script cleans up the dataframe and uses an API and relational algebra to add registrant details to it.

In [2]:
import gzip
import pandas as pd
import json
import requests
from geopy.geocoders import Nominatim

In [4]:
df = pd.read_csv('allADSBx.csv')
df.shape

<ipython-input-4-2f74cc0f619c>:1: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('allADSBx.csv')


(295369, 58)

In [8]:
dfADSBX = df.copy()

business_jet_type_codes = pd.read_csv('business_jet_type_codes.csv')['code'].to_list()

dfADSBX['lat_rnd'],  dfADSBX['lon_rnd'] = dfADSBX['lat'].round(1), dfADSBX['lon'].round(1)
dfADSBX = dfADSBX.loc[dfADSBX['t'].isin(business_jet_type_codes)]

dfADSBX = dfADSBX[dfADSBX['alt_baro'] == "ground"][['hex', 'flight', 'r', 't', 'alt_baro', 'lat_rnd', 'lon_rnd',]].sort_values(by=['lat_rnd','lon_rnd']).drop_duplicates().reset_index(drop=True)

dfADSBX.sample(3)

,hex,flight,r,t,alt_baro,lat_rnd,lon_rnd
395,aadf09,EDG8,N8VC,GLF4,ground,30.2,-97.7
3415,a22e28,NaN,N24FE,CL30,ground,NaN,NaN
796,a2dd33,N284DS,N284DS,GLEX,ground,34.2,-118.5


In [9]:
### Use API to build dataframe of all registered owners 
### This step is donw late in the process, in order to limit number of API requests

def planeRegistrationDetails(icao_hex_codes):
    
    df = pd.DataFrame()
    
    for icao in icao_hex_codes:
        
        if pd.isna(icao):
            pass
        else:
            api_icaohex = f'https://hexdb.io/api/v1/aircraft/{icao}'
            api_response = requests.get(api_icaohex)
            j = json.loads(api_response.text)
            dfx = pd.DataFrame(j, index=[0])
            df = pd.concat([df, dfx])
        
    df['ModeS'] = df['ModeS'].str.lower()
    df = df.reset_index(drop=True)

    return df

icao_hex_codes = dfADSBX['hex'].unique()
dfREGO = planeRegistrationDetails(icao_hex_codes)
dfREGO.sample(10)

,ICAOTypeCode,Manufacturer,ModeS,OperatorFlagCode,RegisteredOwners,Registration,Type,status,error
297,G280,Gulfstream Aerospace,a2ce8d,G280,Richardson Aviation,N280FW,G280,NaN,NaN
457,CL35,Bombardier,aa6a76,EJA,NetJets,N770QS,Challenger 350,NaN,NaN
896,F900,Dassault,a9a1b9,F900,Obsidian Aircraft LLC,N72FJ,Falcon 900 B,NaN,NaN
1019,G280,Gulfstream Aerospace,a5892f,COO,Cove Aviation,N456JA,G280,NaN,NaN
217,CL35,Bombardier,a95262,CL35,Tabletop LLC,N70BV,Challenger 350,NaN,NaN
1023,GLEX,Bombardier,4056a2,VCG,Catreus,G-CEYL,Global Express XRS,NaN,NaN
326,GL7T,Bombardier,a12d2f,EJA,NetJets,N175QS,Global 7500,NaN,NaN
183,E35L,Embraer,3c564f,AHO,Air Hamburg GmbH,D-AERO,Legacy 650,NaN,NaN
991,GL6T,Bombardier,4948b5,JME,Executive Jet Management (Europe),CS-REU,Global 6000,NaN,NaN
187,CL30,Bombardier,a3fc35,CL30,Corporate,N356ML,Challenger 300,NaN,NaN


In [ ]:
## Use outer merge on dataframes to combine

dfM = dfADSBX.merge(dfREGO, left_on='hex', right_on='ModeS', how="outer")
#dfM = dfM.merge(dfCOORDS, on=['lat_rnd', 'lon_rnd'], how="outer")

dfM = dfM.loc[dfM['status'] != 404]
dfM = dfM[(dfM['hex'].notnull()) & (dfM['flight'].notnull())]
dfM = dfM[(dfM['lat_rnd'].notnull()) & (dfM['lon_rnd'].notnull())]

dfM['RegisteredOwners'] = dfM['RegisteredOwners'].fillna('---')
dfM['Manufacturer'] = dfM['Manufacturer'].fillna('---')
dfM['Type'] = dfM['Type'].fillna('---')

dfM = dfM[[x for x in dfM.columns if x not in ['ModeS', 'error', 'status', 'Registration']]]
dfM = dfM.sort_values(by=['lat_rnd', 'lon_rnd']).reset_index(drop=True)
dfM.to_csv('dfMERGE.csv', index = False)
dfM.sample(20)